# NuGetConsume

NGC is an API to depend on NuGet packages in Dyalog APL.

It takes care of installing from NuGet and some of the managing of `⎕USING` statements.

`⎕using←'NSName,[thing]'`
thing is either name of Assembly or path to file

the bridge can call Assembly.Load and "lots of the time?" do the gymnastics to bring in the assembly

## Features
- add packages
- get path to .dll for `⎕USING` (additional namespaces might not be possible automatically)
- maybe how to export .csproj?
    - or just list of .NET/NuGet dependencies as part of ]Cider/Tatin config?
- does NGC need stuff for when we export APL as .NET stuff?
- `NGC.Export'publish'` will export DLLs
    - `NGC.Export'publish' 'platform'`
- `NGC.GetPath` (`FullPath` in nugetconsum)
- retarget existing project? TBD - might only be able to take packages list and attempt create new .NET project

## How to ship APL as a (NuGet/.NET) library
- provide bridge and other DLLs/exes etc. as dependencies
- `⎕NQ'Bind'` for export assembly needs dependency info

#todo shipping APL generally:
- end user non-APLer app
- C# dev
- APL dev e.g. UCMD

## Todo
- check can `⎕NEW` on existing .csproj
- document info on platform compatibility
- can we get DLL name from package info? esp. if different from package name...
- can we get assembly namespaces from package info (I think Bjørn said not necessarily)
- tmp path for .csproj by default?
- can we have it *not* be a class?
    - `⎕NEW thing` to carry around info seems nice
    - but end up not hiding anything since all my methods are public?
    - but what other way to hold the info? pass a namespace around to functions? 
- do we need `Hook/NuGetPackageHandler` methods if we do dependencies from `project.assets.json`?
- should user supply Assebly Namespaces at `AddPackage` or when setting `⎕USING`?
- `⎕NULL in [3] will add & install the package, but not add it to ⎕USING` ?
- can we get chromedriver exe location from package info?

## .NET projects
NGC uses the `dotnet` Command Line Interface (CLI) to set up a .NET project which manages obtaining compatible versions of NuGet packages and their dependencies.

The project is defined in a **.csproj** file, while dependencies and paths to assembly DLLs are listed in **obj/project.assets.json**.

## .NET Versions
Dyalog can be configured to use various .NET versions [*Dyalog Forums post*](https://forums.dyalog.com/viewtopic.php?f=22&t=1863)

> kind of bleh  
> Developers need to install the **SDK** in order to target dotnet .csproj projects  
> alternative?  
> learn the nuget API and manage deps ourselves... also bleh

**Officially supported .NET versions:**
|Dyalog version|.NET version|.NET Framework (Windows only)
|---|---|---|
|17.1|3.1|4.8|
|18.2|3.1|4.8|
|19.0|6.0|4.8|

NGC will create and export .NET packages under a single .NET project targeting the currently running .NET version.

NuGet Gallery package pages ([example: Clock](https://www.nuget.org/packages/Clock#supportedframeworks-body-tab)) list compatible versions under "frameworks" tab. **netstandard** is supported across .NET versions, while **netN** for some number **N** corresponds to a particular .NET version.

> **Note:** `dotnet` CLI seems just NETCore/New .NET or NetStandard - need to force .NET Framework?  
> `dotnet new classlib --target-framework-override net48 -o /tmp/myapp`

## Usage
Create an instance of the NGC class.

In [138]:
]get "C:\g\NuGetConsume\APLSource\NGC.apln"

#.NGC

**Note:** Jupyter kernel doesn't seem to play nice with FSW - no idea why

In [139]:
ngc←⎕NEW NGC'/tmp/myapp'

The template "Class Library" was created successfully. 
 
Processing post-creation actions... 
Running 'dotnet restore' on C:\tmp\myapp\myapp.csproj...
 Determining projects to restore... 
 Restored C:\tmp\myapp\myapp.csproj (in 95 ms). 
Restore succeeded.

***Note:*** Jupyter kernel can't print asynchronous output... [issue 82](https://github.com/Dyalog/dyalog-jupyter-kernel/issues/82)

Or maybe unrelated... why does AddPackages output work?

In [140]:
⎕←ngc.project_dir
⎕←ngc.dotnet_version

/tmp/myapp

net48

In [141]:
ngc.AddPackages 'Clock'

Determining projects to restore...
 Writing C:\Users\rpark\AppData\Local\Temp\tmpD751.tmp
info : Adding PackageReference for package 'Clock' into project 'C:\tmp\myapp\myapp.csproj'.
info : GET https://api.nuget.org/v3/registration5-gz-semver2/clock/index.json
info : OK https://api.nuget.org/v3/registration5-gz-semver2/clock/index.json 137ms
info : Restoring packages for C:\tmp\myapp\myapp.csproj...
info : Package 'Clock' is compatible with all the specified frameworks in project 'C:\tmp\myapp\myapp.csproj'.
info : PackageReference for package 'Clock' version '1.0.3' added to file 'C:\tmp\myapp\myapp.csproj'.
info : Generating MSBuild file C:\tmp\myapp\obj\myapp.csproj.nuget.g.props.
info : Writing assets file to disk. Path: C:\tmp\myapp\obj\project.assets.json
log : Restored C:\tmp\myapp\myapp.csproj (in 107 ms).

In [143]:
⎕USING←''
⎕←↑using←ngc.Using'Clock'
⎕USING,←using
Clock
Clock.UtcNow

Clock,C:\Users\rpark/.nuget/packages/Clock/1.0.3/lib/netstandard1.3/Clock.dll

(Clock)

11/07/2023 14:35:48 +00:00

## Parque.Net

In [144]:
ngc.AddPackages 'Parquet.Net' '4.15.0'

Determining projects to restore...
 Writing C:\Users\rpark\AppData\Local\Temp\tmp407B.tmp
info : Adding PackageReference for package 'Parquet.Net' into project 'C:\tmp\myapp\myapp.csproj'.
info : Restoring packages for C:\tmp\myapp\myapp.csproj...
info : GET https://api.nuget.org/v3-flatcontainer/system.text.json/index.json
info : GET https://api.nuget.org/v3-flatcontainer/system.reflection.emit.lightweight/index.json
info : GET https://api.nuget.org/v3-flatcontainer/microsoft.bcl.asyncinterfaces/index.json
info : GET https://api.nuget.org/v3-flatcontainer/system.numerics.vectors/index.json
info : OK https://api.nuget.org/v3-flatcontainer/microsoft.bcl.asyncinterfaces/index.json 107ms
info : GET https://api.nuget.org/v3-flatcontainer/microsoft.bcl.asyncinterfaces/5.0.0/microsoft.bcl.asyncinterfaces.5.0.0.nupkg
info : OK https://api.nuget.org/v3-flatcontainer/system.numerics.vectors/index.json 133ms
info : GET https://api.nuget.org/v3-flatcontainer/system.numerics.vectors/4.4.0/system.numerics.vectors.4.4.0.nupkg
info : OK https://api.nuget.org/v3-flatcontainer/system.text.json/index.json 151ms
info : OK https://api.nuget.org/v3-flatcontainer/system.reflection.emit.lightweight/index.json 155ms
info : GET https://api.nuget.org/v3-flatcontainer/system.text.json/7.0.2/system.text.json.7.0.2.nupkg
info : OK https://api.nuget.org/v3-flatcontainer/microsoft.bcl.asyncinterfaces/5.0.0/microsoft.bcl.asyncinterfaces.5.0.0.nupkg 41ms
info : OK https://api.nuget.org/v3-flatcontainer/system.numerics.vectors/4.4.0/system.numerics.vectors.4.4.0.nupkg 27ms
info : GET https://api.nuget.org/v3-flatcontainer/system.reflection.emit.lightweight/4.7.0/system.reflection.emit.lightweight.4.7.0.nupkg
info : OK https://api.nuget.org/v3-flatcontainer/system.text.json/7.0.2/system.text.json.7.0.2.nupkg 34ms
info : OK https://api.nuget.org/v3-flatcontainer/system.reflection.emit.lightweight/4.7.0/system.reflection.emit.lightweight.4.7.0.nupkg 51ms
info : GET https://api.nuget.org/v3-flatcontainer/microsoft.bcl.asyncinterfaces/7.0.0/microsoft.bcl.asyncinterfaces.7.0.0.nupkg
info : GET https://api.nuget.org/v3-flatcontainer/system.text.encodings.web/index.json
info : OK https://api.nuget.org/v3-flatcontainer/microsoft.bcl.asyncinterfaces/7.0.0/microsoft.bcl.asyncinterfaces.7.0.0.nupkg 22ms
info : OK https://api.nuget.org/v3-flatcontainer/system.text.encodings.web/index.json 112ms
info : GET https://api.nuget.org/v3-flatcontainer/system.text.encodings.web/7.0.0/system.text.encodings.web.7.0.0.nupkg
info : OK https://api.nuget.org/v3-flatcontainer/system.text.encodings.web/7.0.0/system.text.encodings.web.7.0.0.nupkg 15ms
info : Installed Microsoft.Bcl.AsyncInterfaces 7.0.0 from https://api.nuget.org/v3/index.json with content hash 3aeMZ1N0lJoSyzqiP03hqemtb1BijhsJADdobn/4nsMJ8V1H+CrpuduUe4hlRdx+ikBQju1VGjMD1GJ3Sk05Eg==.
info : Installed System.Text.Encodings.Web 7.0.0 from https://api.nuget.org/v3/index.json with content hash OP6umVGxc0Z0MvZQBVigj4/U31Pw72ITihDWP9WiWDm+q5aoe0GaJivsfYGq53o6dxH7DcXWiCTl7+0o2CGdmg==.
info : Installed Microsoft.Bcl.AsyncInterfaces 5.0.0 from https://api.nuget.org/v3/index.json with content hash W8DPQjkMScOMTtJbPwmPyj9c3zYSFGawDW3jwlBOOsnY+EzZFLgNQ/UMkK35JmkNOVPdCyPr2Tw7Vv9N+KA3ZQ==.
info : Installed System.Reflection.Emit.Lightweight 4.7.0 from https://api.nuget.org/v3/index.json with content hash a4OLB4IITxAXJeV74MDx49Oq2+PsF6Sml54XAFv+2RyWwtDBcabzoxiiJRhdhx+gaohLh4hEGCLQyBozXoQPqA==.
info : Installed System.Numerics.Vectors 4.4.0 from https://api.nuget.org/v3/index.json with content hash UiLzLW+Lw6HLed1Hcg+8jSRttrbuXv7DANVj0DkL9g6EnnzbL75EB7EWsw5uRbhxd/4YdG8li5XizGWepmG3PQ==.
info : Installed System.Text.Json 7.0.2 from https://api.nuget.org/v3/index.json with content hash /LZf/JrGyilojqwpaywb+sSz8Tew7ij4K/Sk+UW8AKfAK7KRhR6mKpKtTm06cYA7bCpGTWfYksIW+mVsdxPegQ==.
info : Package 'Parquet.Net' is compatible with all the specified frameworks in project 'C:\tmp\myapp\myapp.csproj'.
info : PackageReference for package 'Parquet.Net' vers

In [146]:
⍝⎕USING←'System.IO,System.IO.FileSystem,System.IO.Stream'
⍪ngc.Using'Parquet.Net'

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/netstandard1.3/Apache.Arrow.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/netstandard2.0/IronCompress.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net462/Microsoft.Bcl.AsyncInterfaces.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/netstandard2.0/Microsoft.Data.Analysis.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net462/Microsoft.IO.RecyclableMemoryStream.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/netstandard2.0/Microsoft.ML.DataView.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net46/Microsoft.Win32.Primitives.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/netstandard2.0/Parquet.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/netstandard2.0/Snappier.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net463/System.AppContext.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net461/System.Buffers.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/netstandard2.0/System.Collections.Immutable.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net46/System.Console.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net46/System.Diagnostics.DiagnosticSource.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net462/System.Diagnostics.Tracing.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net46/System.Globalization.Calendars.dll │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Parquet.Net,C:\Users\rpark/.nuget/packages/Parquet.Net/4.15.0/lib/net462/System.IO.dll │
├────────────────────────────────────────────────

In [148]:
⍝⎕USING←'System.IO,System.IO.dll,System.IO.FileSystem,System.IO.Stream'
⎕USING←'' 'System' 'System.IO'
⎕USING,←ngc.Using'Parquet.Net'
Parquet

Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\netstandard1.3\Apache.Arrow.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\netstandard2.0\IronCompress.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net462\Microsoft.Bcl.AsyncInterfaces.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\netstandard2.0\Microsoft.Data.Analysis.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net462\Microsoft.IO.RecyclableMemoryStream.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\netstandard2.0\Microsoft.ML.DataView.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net46\Microsoft.Win32.Primitives.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\netstandard2.0\Snappier.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net463\System.AppContext.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net461\System.Buffers.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\netstandard2.0\System.Collections.Immutable.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net46\System.Console.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net46\System.Diagnostics.DiagnosticSource.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net462\System.Diagnostics.Tracing.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net46\System.Globalization.Calendars.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net462\System.IO.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net46\System.IO.Compression.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net46\System.IO.Compression.ZipFile.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net46\System.IO.FileSystem.dll' or one of its dependencies. The system cannot find the file specified.
Could not load file or assembly 'file:///C:\Users\rpark\.nuget\packages\Parquet.Net\4.15.0\lib\net46\System.IO.FileSystem.Primitives.dll' or one of i